In [1]:
pip install torchtuples

     |████████████████████████████████| 41 kB 292 kB/s 


In [2]:
pip install pycox

     |████████████████████████████████| 73 kB 2.0 MB/s 
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 121 kB 13.1 MB/s 
     |████████████████████████████████| 1.9 MB 13.1 MB/s 
     |████████████████████████████████| 357 kB 46.3 MB/s 
     |████████████████████████████████| 2.2 MB 30.4 MB/s 


In [3]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 8.3 MB/s 


In [4]:
import xlsxwriter

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

In [6]:
import torch # For building the networks 
import torchtuples as tt # Some useful functions
import pandas as pd
import os


In [7]:
from pycox.datasets import metabric
from pycox.models import LogisticHazard
from pycox.models import PMF
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [9]:
# !ls '/gdrive/My Drive/SurvivalDatasets'

In [10]:
def readDataset(DBName,yourLocation):  
  path = yourLocation+'/'+DBName
  df=pd.read_excel(path  ,sheet_name = 'Data' , engine='openpyxl' ,header=0)
  return df

In [11]:
# np.random.seed(1234)
# _ = torch.manual_seed(123)

In [12]:
# def splitDataset(df):    
#   df_train = df
#   df_test = df_train.sample(frac=0.2)
#   df_train = df_train.drop(df_test.index)
#   df_val = df_train.sample(frac=0.2)
#   df_train = df_train.drop(df_val.index)
#   return [df_train , df_val , df_test ]

In [13]:
# df_train.head() 

In [14]:
# def StandardDataset(columnName , df_train , df_val , df_test ):    
#   cols_standardize = columnName[:-2]
#   cols_leave = ['Time', 'Death']

#   # standardize = [([col], StandardScaler()) for col in cols_standardize]
#   standardize = [([col], None) for col in cols_standardize]
#   leave = [(col, None) for col in cols_leave]

#   x_mapper = DataFrameMapper(standardize + leave)
#   x_train = x_mapper.fit_transform(df_train).astype('float32')
#   x_val = x_mapper.transform(df_val).astype('float32')
#   x_test = x_mapper.transform(df_test).astype('float32')

#   return [x_train ,x_val,x_test ]

In [15]:
def DeleteBlankRows(df , j,columnName , n): 
  df[columnName[n+(2*j)]].replace('', np.nan, inplace=True)
  df[columnName[n+((2*j)+1)]].replace('', np.nan, inplace=True)
  df.dropna(subset=[columnName[n+(2*j)]], inplace=True)
  df.dropna(subset=[columnName[n+((2*j)+1)]], inplace=True)

  return df

In [16]:
def StandardDataset( columnName  ,n,j ,df_train  , df_test , df_val):
  # print(n)    

  cols_standardize = columnName[0:n]
  # print(cols_standardize)
  cols_leave = [columnName[n+(2*j)] , columnName[n+((2*j)+1)]]
  # print(columnName[n+(2*j)])
  # print( columnName[n+((2*j)+1)])

  standardize = [([col], None) for col in cols_standardize]
  leave = [(col, None) for col in cols_leave]

  x_mapper = DataFrameMapper(standardize + leave)

  x_train = x_mapper.fit_transform(df_train).astype('float32')
  x_test = x_mapper.transform(df_test).astype('float32')
  x_val = x_mapper.transform(df_val).astype('float32')

  return [x_train ,x_test , x_val]

In [61]:

nof = 5
############################################
outcome = 'Progression'  ### 'Distant'   'Death'   'Locoregional'   'Progression'
version = 'V4'  ### 'V3'   'V4'   'V2'
METHODnAME = 'PMF'   ####  method name
loc = "/gdrive/My Drive/Survival"
yourLocation = loc+'/Datasets/'+outcome+'/'+version
#yourLocation = '/gdrive/My Drive/Survival/Datasets/'+outcome+'/'+version

############################################
listOfDataset = os.listdir(yourLocation)
Noutcome = 1
listOfDataset

['F4_Survival_ProgV4.xlsx',
 'kernel_pca.xlsx',
 'pca.xlsx',
 'fa.xlsx',
 'tsne.xlsx',
 'sammon.xlsx',
 'isomap.xlsx',
 'diffusion_maps.xlsx',
 'lle.xlsx',
 'mds.xlsx',
 'spe.xlsx',
 'gplvm.xlsx',
 'sne.xlsx',
 'autoencoder.xlsx',
 'sym_sne.xlsx',
 'cfs.xlsx',
 'laplacian.xlsx',
 'fsasl.xlsx',
 'udfs.xlsx',
 'ufsol.xlsx',
 'llcfs.xlsx',
 'lasso.xlsx',
 'mrmr.xlsx',
 'relieff.xlsx']

In [62]:
ll = len(listOfDataset)
DatasetResult = []
####################################################
DatasetPredictMatrix = []
####################################################
for i in range(0 , ll):
  filename = listOfDataset[i]
  df = readDataset(filename,yourLocation) 
  columnName = df.columns

  n = Noutcome*(-2)
  outcomeResult = []
####################################################
  outcomePredictMatrix = []
####################################################
  for j in range(0,Noutcome):




    df = DeleteBlankRows(df,j,columnName , n) 
    df_Predictor = df.iloc[:, 0:n]
    df_Predictor.reset_index(drop=True, inplace=True)
    # print(df_Predictor)
    # print(columnName[n+(2*j)] , columnName[n+((2*j)+1)])
    df_outcome = df[[columnName[n+(2*j)] , columnName[n+((2*j)+1)]]]
    df_outcome.reset_index(drop=True, inplace=True)
    # print(df_outcome)

    # print(df_Predictor.shape)
    # print(df_outcome.shape)
    # kf = KFold(n_splits=nof , random_state=42 , shuffle  = True)
    kf = KFold(n_splits=nof)

    folds_Cindex = []
####################################################
    FoldsPredictMatrix = []
####################################################
    for train_index, test_index in kf.split(df_Predictor):
        X_train, X_test = df_Predictor.iloc[train_index], df_Predictor.iloc[test_index]
        y_train, y_test = df_outcome.iloc[train_index], df_outcome.iloc[test_index]

        # print(X_train)
        # print(y_train)
        # print(X_test)
        # print(y_test)


###############################################
##  delete random seed = 42 from hyperparameters
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
###############################################        




        pdList = [X_train, y_train]  
        df_tr = pd.concat(pdList,axis=1)

        pdList = [X_test, y_test]  
        df_te = pd.concat(pdList,axis=1)

        pdList = [X_val, y_val]  
        df_val = pd.concat(pdList,axis=1)


        # print(X_test.shape)
        # print(y_test.shape)
        # print(X_val.shape)
        # print(y_val.shape)
        # print(X_train.shape)
        # print(y_train.shape)

        [X_train ,X_test , X_val] = StandardDataset(columnName ,n,j, df_tr , df_te , df_val )  

########################## try 5 , 10 , 100  for num_durations
        num_durations = 10
##########################



        labtrans = PMF.label_transform(num_durations)



####################################################
        get_target = lambda df: (df[columnName[n+((2*j)+1)]].values, df[columnName[n+(2*j)]].values)
####################################################



        # get_target = lambda df: (df[columnName[n+(2*j)]].values, df[columnName[n+((2*j)+1)]].values)
        y_train = labtrans.fit_transform(*get_target(df_tr))
        y_val = labtrans.transform(*get_target(df_val))

        # print(df[columnName[n+((2*j)+1)]].values)
        # print(df[columnName[n+(2*j)]].values)

        train = (X_train, y_train)
        val = (X_val, y_val)

        
        durations_test, events_test = get_target(df_te)

        in_features = X_train.shape[1]
        num_nodes = [32, 32]
        out_features = labtrans.out_features
        batch_norm = True
        dropout = 0.1

        net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)

        # optimizer = tt.optim.AdamWR(decoupled_weight_decay=0.01, cycle_eta_multiplier=0.8,cycle_multiplier=2)
        # model = DeepHitSingle(net, optimizer, duration_index=labtrans.cuts)
        model = PMF(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)

        batch_size = 128

        # lrfind = model.lr_finder(X_train, y_train, batch_size, tolerance=50)
        # print(lrfind.get_best_lr())
        # model.optimizer.set_lr(lrfind.get_best_lr())
        # model.optimizer.set_lr(0.01)

        epochs = 100
        callbacks = [tt.cb.EarlyStopping()]
        verbose = False 
        log = model.fit(X_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val)
        # log = model.fit(x_train, y_train, batch_size, epochs, val_data=val)

        # _ = log.plot()
        # print('loss = ',log.to_pandas().val_loss.min())
        model.score_in_batches(val)

        surv = model.predict_surv_df(X_test)
####################################################
        FoldsPredictMatrix.append(surv)
####################################################
        # surv.iloc[:, :5].plot(drawstyle='steps-post')
        # plt.ylabel('S(t | x)')
        # _ = plt.xlabel('Time')

        # surv = model.interpolate(10).predict_surv_df(X_test)
        # surv.iloc[:, :5].plot(drawstyle='steps-post')
        # plt.ylabel('S(t | x)')
        # _ = plt.xlabel('Time')

        ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    
        print('C index = ' , ev.concordance_td('antolini'))
        folds_Cindex.append(ev.concordance_td('antolini'))
    
    
    
    outcomeResult.append(folds_Cindex)
####################################################
    outcomePredictMatrix.append(FoldsPredictMatrix)
####################################################

  print('-------------------------',i,'-------------------------')
  DatasetResult.append(outcomeResult)
####################################################
  DatasetPredictMatrix.append(outcomePredictMatrix)   
#################################################### 

C index =  0.7748607796340493
C index =  0.523371104815864
C index =  0.7984395318595578
C index =  0.5783718104495748
C index =  0.9595701125895599
------------------------- 0 -------------------------
C index =  0.8655529037390612
C index =  0.5495750708215298
C index =  0.5500650195058517
C index =  0.6200891049007695
C index =  0.9012282497441146
------------------------- 1 -------------------------
C index =  0.8591885441527446
C index =  0.7726628895184136
C index =  0.7925877763328999
C index =  0.5091130012150669
C index =  0.6939611054247697
------------------------- 2 -------------------------
C index =  0.711217183770883
C index =  0.5014164305949008
C index =  0.6853055916775033
C index =  0.509518023491292
C index =  0.8137154554759468
------------------------- 3 -------------------------
C index =  0.8560063643595863
C index =  0.703257790368272
C index =  0.7685305591677504
C index =  0.6767922235722965
C index =  0.8950870010235414
------------------------- 4 ----------

In [63]:
##################################################

dirName = loc+ "/Results/"+outcome+"/"+METHODnAME
#dirName = '/gdrive/My Drive/Survival/Datasets/'+outcome+'/'+version

if not os.path.exists(dirName):
  os.mkdir(dirName)

max = 0
index = -1
filepath = dirName+"/Results_"+version+".txt"

file = open(filepath, "w")
for i in range(0 , ll):
  out = str(listOfDataset[i]) + ' -> mean : ' + str(np.mean(DatasetResult[i][0])) + ' -> std : ' + str(np.std(DatasetResult[i][0]))+'\n'
  print(out)
  file.write(out)
  if max < np.mean(DatasetResult[i][0]):
    max = np.mean(DatasetResult[i][0])
    index = i

print('\n\n\n\n')
print('BestResult -->  ' ,listOfDataset[index] , ' -> mean : ' , np.mean(DatasetResult[index][0]) , ' -> std : ' , np.std(DatasetResult[index][0]))
file.close()
##################################################
# np.std(DatasetResult[0][0])
# DatasetResult[dataset][outcome][fold]



F4_Survival_ProgV4.xlsx -> mean : 0.7269226678697212 -> std : 0.15813801187005577

kernel_pca.xlsx -> mean : 0.6973020697422653 -> std : 0.15450442527178648

pca.xlsx -> mean : 0.7255026633287789 -> std : 0.12034597608346627

fa.xlsx -> mean : 0.6442345370021052 -> std : 0.12119640156150348

tsne.xlsx -> mean : 0.7799347876982894 -> std : 0.0844922764421089

sammon.xlsx -> mean : 0.7212294499386926 -> std : 0.07515972339042978

isomap.xlsx -> mean : 0.7658298125400751 -> std : 0.1469787714493384

diffusion_maps.xlsx -> mean : 0.7167760027861729 -> std : 0.18835063415591521

lle.xlsx -> mean : 0.6751121252810443 -> std : 0.14410618363284097

mds.xlsx -> mean : 0.7044933575531421 -> std : 0.15770245845826994

spe.xlsx -> mean : 0.6145578993065574 -> std : 0.09793223752714796

gplvm.xlsx -> mean : 0.6572732737208643 -> std : 0.1331747372414932

sne.xlsx -> mean : 0.7081072221697011 -> std : 0.16303805892474163

autoencoder.xlsx -> mean : 0.6693103487772529 -> std : 0.17376462916393612

sy

In [64]:
####################################################
dirName = loc+ "/Results/"+outcome+"/"+METHODnAME
if not os.path.exists(dirName):
  os.mkdir(dirName)

# pdList = [df1, df2, ...]  # List of your dataframes
# new_df = pd.concat(pdList)


for i in range(0 , ll):
  filepath = dirName+"/ResultsPrediction_"+version+"_"+str(listOfDataset[i][0:-5])+".xlsx"
  DaTaAllFolds = []
  for j in range(0 , nof):
    DaTaAllFolds.append(DatasetPredictMatrix[i][0][j])

  dfEmpty1 = pd.DataFrame(['Fold1'])
  dfEmpty2 = pd.DataFrame(['Fold2'])
  dfEmpty3 = pd.DataFrame(['Fold3'])
  dfEmpty4 = pd.DataFrame(['Fold4'])
  dfEmpty5 = pd.DataFrame(['Fold5'])

  # print(dfEmpty)
  df1 = pd.DataFrame(DaTaAllFolds[0])
  df2 = pd.DataFrame(DaTaAllFolds[1])
  df3 = pd.DataFrame(DaTaAllFolds[2])
  df4 = pd.DataFrame(DaTaAllFolds[3])
  df5 = pd.DataFrame(DaTaAllFolds[4])

  pdList = [dfEmpty1 , df1, dfEmpty2 , df2, dfEmpty3 ,df3, dfEmpty4 ,df4, dfEmpty5 ,df5]  # List of your dataframes
  new_df = pd.concat(pdList) 
  # print(new_df) 
  # df = pd.DataFrame(DaTaAllFolds)
  writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
  new_df.to_excel(writer, sheet_name='ResultsPrediction', index=False)
  writer.save()
####################################################
# DatasetResult[dataset][outcome][fold]